In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("="*80)
print("NYC REAL ESTATE PROJECT - PHASE 0: DATA EXPLORATION")
print("="*80)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Check if dataset exists
data_path = '../data/raw/nyc-property-sales.csv'

if os.path.exists(data_path):
    size_mb = os.path.getsize(data_path) / (1024**2)
    print(f"✓ Dataset found: {data_path}")
    print(f"✓ File size: {size_mb:.2f} MB")
else:
    print(f"✗ ERROR: Dataset NOT found at {data_path}")
    print("Current directory contents:")
    print(os.listdir('../data/raw/'))

In [ ]:
print("\n" + "="*80)
print("LOADING SAMPLE DATA (100,000 rows)")
print("="*80)

import time
start = time.time()

# Load sample
df_sample = pd.read_csv(data_path, nrows=100000)
load_time = time.time() - start

print(f"✓ Loaded {len(df_sample):,} rows in {load_time:.2f} seconds")
print(f"✓ Shape: {df_sample.shape}")
print(f"✓ Memory usage: {df_sample.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

In [ ]:
print("\n" + "="*80)
print("DATASET STRUCTURE")
print("="*80)

print(f"\n📊 Total Columns: {len(df_sample.columns)}\n")

for i, col in enumerate(df_sample.columns, 1):
    dtype = df_sample[col].dtype
    nunique = df_sample[col].nunique()
    missing = df_sample[col].isnull().sum()
    missing_pct = (missing / len(df_sample)) * 100
    
    print(f"{i:2d}. {col:30s} | Type: {str(dtype):10s} | Unique: {nunique:6,} | Missing: {missing:5,} ({missing_pct:5.2f}%)")

In [ ]:
print("\n" + "="*80)
print("FIRST 5 ROWS")
print("="*80)

df_sample.head()

In [ ]:
print("\n" + "="*80)
print("NUMERICAL COLUMNS STATISTICS")
print("="*80)

df_sample.describe()

In [ ]:
print("\n" + "="*80)
print("MISSING VALUES ANALYSIS")
print("="*80)

missing = df_sample.isnull().sum()
missing_pct = (missing / len(df_sample)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_%': missing_pct.values
})

missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(f"\n{len(missing_df)} columns have missing values:\n")
    print(missing_df.to_string(index=False))
else:
    print("\n✓ No missing values found!")

In [ ]:
print("\n" + "="*80)
print("SALE PRICE ANALYSIS - KEY DATA QUALITY ISSUES")
print("="*80)

print("\n📈 Basic Statistics:")
print(df_sample['SALE PRICE'].describe())

print("\n🚨 Data Quality Issues:")
print("─"*60)

# Zero dollar sales
zero_sales = (df_sample['SALE PRICE'] == 0).sum()
zero_pct = (zero_sales / len(df_sample)) * 100
print(f"  • Zero dollar sales:        {zero_sales:6,} ({zero_pct:5.2f}%)")

# Negative prices
negative = (df_sample['SALE PRICE'] < 0).sum()
print(f"  • Negative prices:          {negative:6,} ({negative/len(df_sample)*100:5.2f}%)")

# Low value sales
low_sales = ((df_sample['SALE PRICE'] > 0) & (df_sample['SALE PRICE'] < 100000)).sum()
low_pct = (low_sales / len(df_sample)) * 100
print(f"  • Low value (<$100K):       {low_sales:6,} ({low_pct:5.2f}%)")

# Valid range
valid = ((df_sample['SALE PRICE'] >= 100000) & (df_sample['SALE PRICE'] <= 20000000)).sum()
valid_pct = (valid / len(df_sample)) * 100
print(f"  • Valid ($100K-$20M):       {valid:6,} ({valid_pct:5.2f}%)")

# Extreme outliers
extreme = (df_sample['SALE PRICE'] > 20000000).sum()
extreme_pct = (extreme / len(df_sample)) * 100
print(f"  • Extreme outliers (>$20M): {extreme:6,} ({extreme_pct:5.2f}%)")

print("─"*60)
print(f"  TOTAL:                      {len(df_sample):6,} (100.00%)")

print(f"\n💡 Expected rows after filtering: ~{valid:,} ({valid_pct:.1f}% of data)")

In [ ]:
print("\n" + "="*80)
print("BOROUGH DISTRIBUTION")
print("="*80)

borough_counts = df_sample['BOROUGH'].value_counts().sort_index()
print("\nSales by Borough:")
print(borough_counts.to_string())

print("\nBorough codes:")
print("  1 = Manhattan")
print("  2 = Bronx")
print("  3 = Brooklyn")
print("  4 = Queens")
print("  5 = Staten Island")

In [ ]:
print("\n" + "="*80)
print("CATEGORICAL FEATURES CARDINALITY")
print("="*80)

cat_features = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 
                'BUILDING CLASS AT PRESENT', 'ZIP CODE', 'TAX CLASS AT PRESENT']

print("\n📊 Number of unique values per categorical feature:\n")

for col in cat_features:
    if col in df_sample.columns:
        nunique = df_sample[col].nunique()
        print(f"  • {col:30s}: {nunique:5,} unique values")

print("\n💡 High-cardinality features (NEIGHBORHOOD, ZIP CODE) will benefit most from GPU acceleration!")

In [ ]:
print("\n" + "="*80)
print("SALE DATE ANALYSIS")
print("="*80)

print(f"\nOriginal SALE DATE type: {df_sample['SALE DATE'].dtype}")
print("\nSample values:")
print(df_sample['SALE DATE'].head(10))

# Try converting to see if there are issues
try:
    date_converted = pd.to_datetime(df_sample['SALE DATE'], errors='coerce')
    null_dates = date_converted.isnull().sum()
    print(f"\n✓ Date conversion successful")
    print(f"  Failed conversions: {null_dates:,} ({null_dates/len(df_sample)*100:.2f}%)")
    
    if null_dates == 0:
        print(f"  Date range: {date_converted.min()} to {date_converted.max()}")
except Exception as e:
    print(f"\n✗ Date conversion error: {e}")

In [ ]:
print("\n" + "="*80)
print("CREATING ADDITIONAL VISUALIZATION - PRICE BY BOROUGH")
print("="*80)

# Re-create filtered dataset for this cell
filtered_for_plot = df_sample[(df_sample['SALE PRICE'] >= 100000) & 
                               (df_sample['SALE PRICE'] <= 20000000)].copy()

print(f"Filtered dataset for visualization: {len(filtered_for_plot):,} rows")

# Create boxplot
fig, ax = plt.subplots(figsize=(14, 6))

# Convert borough to string for better labels
filtered_for_plot['BOROUGH'] = filtered_for_plot['BOROUGH'].astype(str)

# Create boxplot
filtered_for_plot.boxplot(column='SALE PRICE', by='BOROUGH', ax=ax)

ax.set_title('Sale Price Distribution by Borough (Filtered: $100K-$20M)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Borough (1=Manhattan, 2=Bronx, 3=Brooklyn, 4=Queens, 5=Staten Island)', 
              fontsize=11)
ax.set_ylabel('Sale Price ($)', fontsize=12)
plt.suptitle('')  # Remove default title

plt.tight_layout()

# Save
os.makedirs('../analysis/plots', exist_ok=True)
plt.savefig('../analysis/plots/phase0_price_by_borough.png', dpi=300, bbox_inches='tight')
print("\n✓ Boxplot saved to: analysis/plots/phase0_price_by_borough.png")
plt.show()

In [ ]:
print("\n" + "="*80)
print("SAVING SAMPLE DATA FOR QUICK TESTING")
print("="*80)

# Save sample
sample_output = '../data/sample/sample_100k.csv'
os.makedirs('../data/sample', exist_ok=True)

df_sample.to_csv(sample_output, index=False)
sample_size = os.path.getsize(sample_output) / (1024**2)

print(f"✓ Sample saved to: {sample_output}")
print(f"✓ Sample size: {sample_size:.2f} MB")

In [ ]:
print("\n" + "="*80)
print("KEY FINDINGS & PREPROCESSING STRATEGY")
print("="*80)

print("\n📋 DATA QUALITY ISSUES IDENTIFIED:")
print("─"*60)
print(f"  1. Zero-dollar sales: {zero_sales:,} rows → REMOVE")
print(f"  2. Low-value sales (<$100K): {low_sales:,} rows → REMOVE")
print(f"  3. Extreme outliers (>$20M): {extreme:,} rows → REMOVE")
if len(missing_df) > 0:
    print(f"  4. Missing values in: {', '.join(missing_df['Column'].head(3).tolist())}")
else:
    print(f"  4. No missing values")

print("\n✅ PREPROCESSING STRATEGY:")
print("─"*60)
print("  1. FILTERING:")
print("     - Remove SALE PRICE = $0 (non-market transfers)")
print("     - Keep only $100,000 ≤ SALE PRICE ≤ $20,000,000")
print("     - Expected result: ~{:,} rows ({:.1f}% of data)".format(valid, valid_pct))

print("\n  2. MISSING VALUES:")
if 'LAND SQUARE FEET' in missing_df['Column'].values:
    print("     - LAND SQUARE FEET: Fill with median")
if 'GROSS SQUARE FEET' in missing_df['Column'].values:
    print("     - GROSS SQUARE FEET: Fill with median")
if 'YEAR BUILT' in missing_df['Column'].values:
    print("     - YEAR BUILT: Fill with median")

print("\n  3. DATE CONVERSION:")
print("     - Convert SALE DATE to datetime format")

print("\n  4. HIGH-CARDINALITY FEATURES (GPU ADVANTAGE):")
print(f"     - NEIGHBORHOOD: {df_sample['NEIGHBORHOOD'].nunique()} categories")

# Check for ZIP CODE column (might have different name/spacing)
zip_col = None
for col in df_sample.columns:
    if 'ZIP' in col.upper():
        zip_col = col
        break

if zip_col:
    print(f"     - {zip_col}: {df_sample[zip_col].nunique()} categories")

# Check for BUILDING CLASS column
building_col = None
for col in df_sample.columns:
    if 'BUILDING CLASS AT PRESENT' in col:
        building_col = col
        break

if building_col:
    print(f"     - {building_col}: {df_sample[building_col].nunique()} categories")

print("\n💡 EXPECTED PERFORMANCE:")
print("─"*60)
print("  • CPU Serial preprocessing: ~20-25 minutes")
print("  • GPU Single preprocessing: ~3-4 minutes (6-8x speedup)")
print("  • Biggest GPU gains: Feature engineering with location aggregations")

print("\n" + "="*80)
print("✓ PHASE 0 EXPLORATION COMPLETE!")
print("="*80)


In [ ]:
# Create exploration summary JSON
exploration_summary = {
    'phase': 'Phase 0 - Exploration',
    'sample_size': len(df_sample),
    'dataset_info': {
        'total_columns': len(df_sample.columns),
        'numerical_columns': len(df_sample.select_dtypes(include=[np.number]).columns),
        'categorical_columns': len(df_sample.select_dtypes(exclude=[np.number]).columns)
    },
    'data_quality': {
        'zero_sales': int(zero_sales),
        'zero_sales_pct': float(zero_pct),
        'low_value_sales': int(low_sales),
        'low_value_pct': float(low_pct),
        'valid_sales': int(valid),
        'valid_sales_pct': float(valid_pct),
        'extreme_outliers': int(extreme),
        'extreme_outliers_pct': float(extreme_pct)
    },
    'missing_values': {
        'columns_with_missing': len(missing_df),
        'details': missing_df.to_dict('records') if len(missing_df) > 0 else []
    },
    'preprocessing_plan': {
        'filter_strategy': 'Keep $100K <= SALE PRICE <= $20M',
        'expected_rows_after_filter': int(valid),
        'missing_value_strategy': 'Fill with median',
        'date_conversion': 'Convert SALE DATE to datetime'
    }
}

import json
os.makedirs('../results/timings', exist_ok=True)
with open('../results/timings/phase0_exploration_summary.json', 'w') as f:
    json.dump(exploration_summary, f, indent=4)

print("✓ Exploration summary saved to: results/timings/phase0_exploration_summary.json")
print("\n🎉 Phase 0 Complete! Ready for Phase 1.")